In [ ]:
__author__ = "Jose David Marroquin Toledo"
__credits__ = ["Jose David Marroquin Toledo", ]
__email__ = "jose@marroquin.cl"
__status__ = "Development"

In [ ]:
from math import cos, sin, radians
import serial
import time
import matplotlib.pyplot as plt
import cv2
from pathlib import Path, PurePath
from datetime import date

In [ ]:
def get_pixels_xy(**kwargs):
    """Calculates the (x, y) coordinates of the Pixels in a ring or
    Jewel and stores them a list of tuple within a dictionary.
    """
    rings = kwargs.pop('rings', {7: 0, 12: 0, 24: 0});
    cm = kwargs.pop('cm', True)  # (x, y) in centimeters?
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(generate_lores_set.__name__,
                  list(kwargs.keys())[-1]))
    # The properties for each NeoPixel ring or Jewel were retrieved from
    # https://learn.adafruit.com/adafruit-neopixel-uberguide/downloads
    NEOPIXEL_LED_RINGS = {
        '7': {
            'name': 'NeoPixel Jewel',
            'diameter': {
                'outer': 0.9,
                'px': 0.63
            },
            'type': 'jewel'
        },
        '12': {
            'name': 'NeoPixel 12-LED Ring',
            'diameter': {
                'inner': 0.92,
                'outer': 1.45,
                'px': 1.16
            },
            'type': 'ring'
        },
        '16': {
            'name': 'NeoPixel 16-LED Ring',
            'diameter': {
                'inner': 1.25,
                'outer': 1.75,
                'px': 1.49
            },
            'type': 'ring'
        },
        '24': {
            'name': 'NeoPixel 24-LED Ring',
            'diameter': {
                'inner': 2.06,
                'outer': 2.58,
                'px': 2.3
            },
            'type': 'ring'
        }
    }
    INCH_CM = 2.54

    pixels_xy = dict()
    for key in rings:
        try:
            print('<RingValid-' + str(key) + '>')
            ring = NEOPIXEL_LED_RINGS[str(key)]
            angle = 0
            ring_xy_list = list()
            outer_pixels = int(key)
            if ring['type'] == 'jewel':
                ring_xy_list.append((0, 0))  # Add the central Pixel.
                outer_pixels -= 1
            dist_px_px = 360 / float(outer_pixels)  # In degree.
            phase = float(rings[key])
            radius = ring['diameter']['px'] / 2.0
            for i in range(outer_pixels):
                pixel_x = cos(radians(angle + phase)) * radius
                pixel_y = sin(radians(angle + phase)) * radius
                if cm:
                    pixel_x = INCH_CM * pixel_x
                    pixel_y = INCH_CM * pixel_y
                xy = tuple([round(pixel_x, 2), round(pixel_y, 2)])
                angle += dist_px_px
                ring_xy_list.append(xy)
            pixels_xy[str(key)] = ring_xy_list
        except KeyError:
            print('<RingInvalid-' + str(key) + '>')
    return pixels_xy

In [ ]:
def take_photos(cam, n_photos,  **kwargs):
    """Illuminates a sample under a microscope with different incident
    angles from a NeoPixel lamp connected to an Arduino and takes a
    photo for each illumination.
    """
    baud_rate = kwargs.pop('baud', 9600)  # In bits per second (baud).
    brightness_level = kwargs.pop('brightness', 255)
    # Time in seconds between a shot and the off of a Pixel.
    time_next_photo = kwargs.pop('cam_wait', 0)
    img_prefix = kwargs.pop('prefix', 'IMG_')
    img_format = kwargs.pop('format', 'PNG')
    outpath = kwargs.pop('path', '')
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(generate_lores_set.__name__,
                  list(kwargs.keys())[-1]))
    # List the serial ports.
    serial_ports = !python -m serial.tools.list_ports
    for i in range(len(serial_ports)):
        # For each line of the output, remove the whitespace characters at
        # the beginning and the end of the string.
        serial_ports[i] = serial_ports[i].strip()
    for item in serial_ports:
        if "ports found" not in item.lower():
            try:
                ser = serial.Serial(item, baud_rate)
                print('<DeviceFound-' + item + '>')
                print('<PC-ArduinoReset>')
                # http://forum.arduino.cc/index.php?topic=38981.0
                # https://stackoverflow.com/questions/21073086/wait-on-arduino-auto-reset-using-pyserial
                ser.dtr = False
                time.sleep(1)
                ser.reset_input_buffer()
                ser.dtr = True
                break
            except serial.SerialException:
                print('<DeviceNotFound>')
    cap = cv2.VideoCapture(cam)
    try:
        if not cap.isOpened():
            # Try to open the cam again.
            if not cap.open(cam):
                # Force to occurr NameError exception is the cam was not
                # opened.
                print('<CamNotFound-' + str(cam) + '>')
                raise NameError
        else:
            print('<CamFound-' + str(cam) + '>')
        ser
        # Make the project's directory.
        home_path = Path.home()
        today_date = str(date.today())
        c = 1
        if outpath != '':
            parent_path = Path(outpath)
        else:
            parent_path = PurePath(home_path, 'SuperScanner', 'proyects',
                                   today_date)
        while True:
            this_project_path = PurePath(parent_path, str(c)).as_posix()
            try:
                Path(this_project_path).mkdir(parents=True)
                print('<DirectoryCreated>')
                print(this_project_path)
                break
            except FileExistsError:
                c = c + 1
        while True:
            try:
                line = ser.readline().strip()
                if b'<ArduinoReady>' in line:
                    print(line.decode())
                    for i in range(n_photos):
                        outcoming_string = (str(i + 1) + ',' +
                                            str(brightness_level))
                        ser.write(outcoming_string.encode())
                        print('<PC-PixelOn-' + str(i + 1) + '>')
                        pixel_on_event = ('<Arduino-PixelOn-' +
                                          str(i + 1) + '>')
                        # https://stackoverflow.com/questions/38645060/what-is-the-equivalent-of-serial-available-in-pyserial
                        while ser.in_waiting == 0:
                            line = ser.readline().strip()
                            if line == pixel_on_event.encode():
                                print(line.decode())
                                break
                        if not cap.isOpened():
                            cap.open(cam)
                        ret, frame = cap.read()
                        img_idx = str(i + 1)
                        for j in range((len(str(n_photos)) -
                                        len(str(i + 1)))):
                            img_idx = '0' + img_idx
                        filename = (img_prefix + img_idx +  '.' +
                                    img_format.lower())
                        file_path = this_project_path + '/' + filename
                        print('<ImageWrite-' + img_idx + '>')
                        cv2.imwrite(file_path, frame)  # Write the photo.
                        # Open the photo below.
                        img = cv2.imread(file_path,
                                         cv2.IMREAD_GRAYSCALE)
                        # Create a figure.
                        plt.figure()
                        try:
                            print('<ImageShow-' + img_idx + '>')
                            plt.imshow(img, cmap="Greys_r")
                        except TypeError:
                            print('<ImageNone-' + str(i) + '>')
                        # Ensure that the photo was taken with light.
                        time.sleep(time_next_photo)
                        # Turn the current Pixel off.
                        outcoming_string = str(i + 1) + ',0'
                        ser.write(outcoming_string.encode())
                        print('<PC-PixelOff-' + str(i + 1) + '>')
                        pixel_off_event = ('<Arduino-PixelOff-' +
                                           str(i + 1) + '>')
                        while ser.in_waiting == 0:
                            line = ser.readline().strip()
                            if line == pixel_off_event.encode():
                                print(line.decode())
                                break
                    raise KeyboardInterrupt
                    break
            except KeyboardInterrupt:
                # https://stackoverflow.com/questions/3208566/nested-exceptions
                raise KeyboardInterrupt('1')
            except serial.SerialException:
                cap.release()
                print('<VideoCaptureClose>')
                break
    except NameError:
        print('<MicroscopeNotAvailable>')
    except KeyboardInterrupt:
        ser.close()
        print('<DeviceClose>')
        cap.release()
        print('<VideoCaptureClose>')